In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 1. Genre-based Recommendation Engine
1. Determine features of a song, and which columns are considered obsolete for decision-making
2. Cluster based on the predetermined features to deterime `genre`
3. Analyze the data based on the new clusters and label them as genres. List all the unique genres.
4. Apply PCA in order to project the newfound clusters into two-dimensional space.
5. Take an input which is one of the unique genres in step 3. Return random 10 songs of that genre.

## Analyzing the sourced raw data
We import the data and see the columns in the .csv file, as well as its corresponding data types (string or numerics). Via the Spotify official developer website, we can understand and take not the overall representation for each column (i.e. what each column means and how they relate to each other)

In [18]:
raw_data = pd.read_csv("data/data.csv")
raw_data.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.604,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.990,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928


In [19]:
raw_data.info() # To see whether the numerical data are already represented as numerics and not strings

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169909 entries, 0 to 169908
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   acousticness      169909 non-null  float64
 1   artists           169909 non-null  object 
 2   danceability      169909 non-null  float64
 3   duration_ms       169909 non-null  int64  
 4   energy            169909 non-null  float64
 5   explicit          169909 non-null  int64  
 6   id                169909 non-null  object 
 7   instrumentalness  169909 non-null  float64
 8   key               169909 non-null  int64  
 9   liveness          169909 non-null  float64
 10  loudness          169909 non-null  float64
 11  mode              169909 non-null  int64  
 12  name              169909 non-null  object 
 13  popularity        169909 non-null  int64  
 14  release_date      169909 non-null  object 
 15  speechiness       169909 non-null  float64
 16  tempo             16

From [this page](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/), we can derive the meaning for each column above. I'll write them down in this cell so we won't need to look back and forth on that page.
1. `Acousticness` represents a confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
2. `Danceability` represents how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. 1.0 is most danceable. 
3. `Energy` represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. 1.0 is most energetic
4. `Instrumentalness` represents the instrumental frequency in the track. 1.0 represents tracks with no vocals.
5. `Liveness` represents the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live
6. `Loudness` represents the loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Values typical range between -60 and 0 db. 
7. `Speechiness` represents the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 are tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks
8. `Valence` represents the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric).
9. `Tempo` represents the overall estimated tempo of a track in beats per minute (BPM).
10. `Mode` 0 = Minor, 1 = Major
11. `Explicit` 0 = No explicit content, 1 = Explicit content\
12. `Key` represents the estimated overall key of the track. Integers map to pitches using standard Pitch Class notation . E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.
---
Despite the data has been cleaned, we can double-check whether a column has any missing `NaN` data. It should return the column name along with the number of missing data besides it.

In [20]:
for colname in raw_data.columns.tolist():
    print("Column {}".format(colname) + " has {} missing values.".format(raw_data[colname].isnull().sum()))

Column acousticness has 0 missing values.
Column artists has 0 missing values.
Column danceability has 0 missing values.
Column duration_ms has 0 missing values.
Column energy has 0 missing values.
Column explicit has 0 missing values.
Column id has 0 missing values.
Column instrumentalness has 0 missing values.
Column key has 0 missing values.
Column liveness has 0 missing values.
Column loudness has 0 missing values.
Column mode has 0 missing values.
Column name has 0 missing values.
Column popularity has 0 missing values.
Column release_date has 0 missing values.
Column speechiness has 0 missing values.
Column tempo has 0 missing values.
Column valence has 0 missing values.
Column year has 0 missing values.


In [21]:
df = raw_data[['id', 'name', 'artists', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'explicit', 'key', 'liveness', 'loudness', 'mode', 'popularity','speechiness', 'tempo', 'valence']]

## Begin to cluster the data
Before we instantiate the clustering on our feature set, we need to determine which value of k is optimal using the elbow method.

We can see that the elbow resides somewhere between `k = 20` clusters. Therefore, we should choose 20 as the number of unique `genres`

In [22]:
from sklearn.cluster import KMeans # Clustering library
%matplotlib inline

features = df[list(df.columns)[3:]]
distortions = []

plt.clf()
k = 20
k_means = KMeans(init = "k-means++", n_clusters = k, n_init = 12) # Run 12 times with different centroid seeds
k_means.fit(features.values)

KMeans(n_clusters=20, n_init=12)

<Figure size 432x288 with 0 Axes>

In [23]:
temp = pd.DataFrame(k_means.labels_)
df = pd.concat([df, temp], axis=1)
df.rename(columns={0: "cluster"}, inplace=True)
df.head()

,id,name,artists,acousticness,danceability,energy,instrumentalness,explicit,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,cluster
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],0.995,0.708,0.1950,0.563,0,10,0.1510,-12.428,1,0,0.0506,118.469,0.7790,12
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",0.994,0.379,0.0135,0.901,0,8,0.0763,-28.454,1,0,0.0462,83.972,0.0767,9
2,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],0.604,0.749,0.2200,0.000,0,5,0.1190,-19.924,0,0,0.9290,107.177,0.8800,12
3,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],0.995,0.781,0.1300,0.887,0,1,0.1110,-14.734,0,0,0.0926,108.003,0.7200,12
4,6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",0.990,0.210,0.2040,0.908,0,11,0.0980,-16.829,1,1,0.0424,62.149,0.0693,9


In [35]:
import ast
df_new = df.copy(deep=True)

In [36]:
df_new['artists'] = df_new['artists'].apply(lambda x: (ast.literal_eval(x)[0]))

In [40]:
for i in range(k):
    to_print = df_new[df_new['cluster'] == i]
    print("Cluster {}".format(i))
    display(to_print.head())
    print("----------------------------------------------------------------------------------------------------")

Cluster 0


,id,name,artists,acousticness,danceability,energy,instrumentalness,explicit,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,cluster
1763,6M2da2eRNj08QJ4TT5vYuU,The Man I Love - 1995 Digital Remaster,Peggy Lee,0.968,0.262,0.2190,0.004390,0,10,0.158,-13.534,0,19,0.0323,112.194,0.118,0
1764,6SEDXcebrNoYClqPORDe91,All Too Soon,Ella Fitzgerald,0.854,0.429,0.1570,0.000000,0,1,0.104,-14.926,1,16,0.0354,125.212,0.209,0
1770,76aRzC9R2XaYOY09k3JfDX,Easy Living,John Lewis,0.994,0.526,0.0872,0.003940,0,8,0.130,-13.066,1,18,0.0446,129.201,0.401,0
1771,79uBIicCUMDSRGomaKI79J,Metaphor,Yusef Lateef,0.847,0.448,0.2020,0.041200,0,2,0.107,-19.357,0,17,0.0429,120.777,0.281,0
1777,0O6GI9vbKCe9kj5hj4hSBf,Right String but the Wrong Yo Yo,Carl Perkins,0.284,0.613,0.8880,0.000051,0,0,0.238,-7.684,1,16,0.0628,117.759,0.889,0


----------------------------------------------------------------------------------------------------
Cluster 1


,id,name,artists,acousticness,danceability,energy,instrumentalness,explicit,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,cluster
33,6Z6DJ8L7WMy2WHbXe3jR9I,A Shropshire Lad: Oh Fair Enough are Sky and P...,George Butterworth,0.987,0.238,0.0324,0.000986,0,7,0.112,-24.361,0,0,0.0541,172.321,0.0379,1
81,6yanEXQBzoNKlWJzKGBpsb,Reaching for Someone (And Not Finding Anyone T...,Paul Whiteman,0.993,0.523,0.0867,0.124000,0,10,0.140,-17.528,1,1,0.0707,161.687,0.6970,1
101,3yZj8i9TwsWAVmBWKdMbaa,Cobardia - Remasterizado,Ignacio Corsini,0.991,0.545,0.1330,0.444000,0,7,0.115,-20.530,0,0,0.1270,185.062,0.6910,1
114,40jiorcB4ujhNCJEM3NYRz,Cantando Bajo la Lluvia - Remasterizado,Francisco Canaro,0.989,0.445,0.2880,0.758000,0,2,0.111,-8.925,1,0,0.0699,205.423,0.6970,1
118,41fXcqGebIUYOFQvDzqLJG,A Lo Lejos - Remasterizado,Francisco Canaro,0.990,0.518,0.4620,0.850000,0,4,0.157,-9.632,0,0,0.3770,178.880,0.6960,1


----------------------------------------------------------------------------------------------------
Cluster 2


...

----------------------------------------------------------------------------------------------------
Cluster 18


,id,name,artists,acousticness,danceability,energy,instrumentalness,explicit,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,cluster
2628,7lN2ZGdAmVQFYvfBhrv0yK,Desesperadamente,Eydie Gormé,0.728,0.537,0.415,0.000000,0,8,0.242,-12.961,1,37,0.0338,112.958,0.833,18
2675,5yQgbjGfbj2U02m3M2wX3N,Stand by Your Man,Tammy Wynette,0.798,0.514,0.326,0.000760,0,9,0.153,-9.690,1,44,0.0291,106.504,0.559,18
2994,3GuUtxubpUewt8tckD79Bl,La Mujer Que Yo Quiero,Joan Manuel Serrat,0.613,0.551,0.492,0.000000,0,1,0.103,-7.188,0,41,0.0288,116.490,0.733,18
3068,21YtaaAivXcpcFaYbTjgKP,To Be Young Gifted and Black,Bob & Marcia,0.418,0.646,0.454,0.000532,0,0,0.353,-10.821,1,41,0.0308,117.125,0.922,18
3072,5FD1IZKwXVSL3zGwlNvLoF,Celoso,Roberto Luti,0.840,0.526,0.501,0.000000,0,0,0.139,-8.088,1,39,0.0367,113.546,0.762,18


----------------------------------------------------------------------------------------------------
Cluster 19


,id,name,artists,acousticness,danceability,energy,instrumentalness,explicit,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,cluster
1688,6443igXMStgLhDvCveDZfQ,Lo Que Quiera Lola,Bobby Capo,0.741,0.782,0.485,0.000122,0,0,0.3340,-11.593,1,19,0.1490,133.127,0.799,19
1759,5zTLkmFCtatsL66leWDzYu,"Italian Concerto in F Major, BWV 971: III. Pre...",Johann Sebastian Bach,0.978,0.263,0.339,0.883000,0,5,0.1320,-19.828,1,19,0.0323,141.828,0.902,19
1772,7Ip6RxnfiDWXO3kp4FAzlO,De Ti Enamorado,La Sonora Matancera,0.828,0.742,0.530,0.000000,0,10,0.0497,-11.410,1,24,0.0409,137.160,0.973,19
1817,66zvey9033Ttmtj09VfwF0,Exactly Like You,Carmen McRae,0.601,0.694,0.353,0.000000,0,5,0.1050,-7.713,1,21,0.0437,135.258,0.563,19
1843,2DHSI9LXjHAdPT8gc5ghbf,Proud Of You,Eddie Cochran,0.678,0.506,0.420,0.000000,0,1,0.2020,-9.640,1,18,0.0481,141.016,0.859,19


----------------------------------------------------------------------------------------------------


In [43]:
df_new[df_new['artists'] == 'Bobby Capo']

,id,name,artists,acousticness,danceability,energy,instrumentalness,explicit,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,cluster
1688,6443igXMStgLhDvCveDZfQ,Lo Que Quiera Lola,Bobby Capo,0.741,0.782,0.485,0.000122,0,0,0.3340,-11.593,1,19,0.1490,133.127,0.799,19
9805,0VntiMue6qOcWmJTDhCKXF,Luna De Miel En Puerto Rico,Bobby Capo,0.834,0.746,0.394,0.003930,0,8,0.1190,-12.410,1,10,0.1450,131.573,0.826,6
71596,0JaPsiY4rZB6g3dGoeYmiR,El Cucú,Bobby Capo,0.817,0.644,0.383,0.000000,0,2,0.0691,-8.933,0,3,0.1790,95.119,0.804,10
137875,2M3YIKtGu9e3SQDcOdtPTB,Me Lo Dijo Adela,Bobby Capo,0.774,0.848,0.593,0.000482,0,3,0.1030,-11.769,0,17,0.1200,135.878,0.963,6
163416,2RaXEKmOuAwQ1NAXbsdbwU,Locamente Enamorado,Bobby Capo,0.867,0.664,0.367,0.000231,0,4,0.3820,-10.599,0,16,0.0501,113.680,0.867,0
